# Lab 1.2 Using Boltz-1 model to predict protein structure

In [1]:
import os
import shutil
import numpy as np

## Helper functions

In [2]:

def preprare_directory(temp, delete_old=True):
    """
    Create a new directory and delete the old one if it exists
    :param temp: str: path to the directory
    :param delete_old: bool, whether to delete the old directory. Defaults to True.
    """
    if delete_old:  
        if os.path.exists(temp):
            # Remove the directory and all its contents
            shutil.rmtree(temp)
    # Recreate the directory
    os.makedirs(temp, exist_ok=True)

## Preparing the input file

[Boltz-1](https://github.com/jwohlwend/boltz/tree/main) is a state-of-the-art open-source model that predicts the 3D structure of proteins, RNA, DNA, and small molecules. It handles modified residues, covalent ligands and glycans, as well as condition the generation on pocket residues.

In this lab, we will primarily look at protein structure prediction. 2 exmaple fils are provided in the `notebooks/input` directory for monomer and multimer. Let's look at the multimer example first: 


```yaml
# notebooks/input/multimer.yaml
version: 1  # Optional, defaults to 1
sequences:
  - protein:
      id: A
      sequence: MAHHHHHHVAVDAVSFTLLQDQLQSVLDTLSEREAGVVRLRFGLTDGQPRTLDEIGQVYGVTRERIRQIESKTMSKLRHPSRSQVLRDYLDGSSGSGTPEERLLRAIFGEKA
  - protein:
      id: B
      sequence: MRYAFAAEATTCNAFWRNVDMTVTALYEVPLGVCTQDPDRWTTTPDDEAKTLCRACPRRWLCARDAVESAGAEGLWAGVVIPESGRARAFALGQLRSLAERNGYPVRDHRVSAQSA
```

To predict the structure, we will use the `boltz predict` command. 
Additional information about the command can be found using: 

```bash
! boltz predict --help
```

In [3]:
! boltz predict --help

Usage: boltz predict [OPTIONS] DATA

  Run predictions with Boltz-1.

Options:
  --out_dir PATH               The path where to save the predictions.
  --cache PATH                 The directory where to download the data and
                               model. Default is ~/.boltz.
  --checkpoint PATH            An optional checkpoint, will use the provided
                               Boltz-1 model by default.
  --devices INTEGER            The number of devices to use for prediction.
                               Default is 1.
  --accelerator [gpu|cpu|tpu]  The accelerator to use for prediction. Default
                               is gpu.
  --recycling_steps INTEGER    The number of recycling steps to use for
                               prediction. Default is 3.
  --sampling_steps INTEGER     The number of sampling steps to use for
                               prediction. Default is 200.
  --diffusion_samples INTEGER  The number of diffusion samples to use for
          

Now let's run the prediction with the following parameters: 

- input file: `input/multimer.yaml`
- `out_dir`: Output directory to stre the result. 
- `devices`: number of GPUs. you can run `nvidia-smi` to find out how many GPUs. In here we have 1
- `output_format`: Save the result in PDB format. 
- `use_msa_server`: Use the MSA server (Colab) to get the MSA. You can optionally provide your own MSA file, but for simplicity, we will use the server here. 

In [7]:
! boltz predict input/adalimumab.yaml --out_dir output --devices 1 --output_format pdb --use_msa_server

Checking input data.
Running predictions for 1 structure
Processing input data.
  0%|                                                     | 0/1 [00:00<?, ?it/s]Generating MSA for input/adalimumab.yaml with 2 protein entities.

COMPLETE: 100%|██████████████████████| 300/300 [elapsed: 00:00 remaining: 00:00]

100%|█████████████████████████████████████████████| 1/1 [00:03<00:00,  3.31s/it]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/sagemaker-user/.conda/envs/workshop/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[ext

The first time this command it is run, it will download the model weights and cache them, and can take a while (~15 min, depending on the network speed). The second time it is run, it will read the cached weights, so it will be much faster. For the example multimer, it should take ~2 minutes to complete on a A10 GPU

## Visualizing the results

In the `notebooks/utput/boltz_results_multimer/predictions/multimer` directory, you should see: 
-  `multimer_model_0.pdb` file: predicted structure
- `confidence_multimer_model_0.json` file: confidence scores for the predicted structure
- `plddt_multimer_model_0.npz` file: per-residue pLDDT confidence scores

Similarly to ESMFold, assume we want to visualize the predicted structure, coloring the residues based on the pLDDT scores. To do this we will first need to extract the pLDDT scores. 

In [ ]:
# Load the .npz file that contains the pLDDT scores
npz_file_path = "output/boltz_results_multimer/predictions/multimer/plddt_multimer_model_0.npz"
data = np.load(npz_file_path)
# extract the pLDDT scores
plddt_scores = data['plddt']

# how many scores are there?
print(plddt_scores.shape)

Note the length of the PLDDt scores = length if the input sequence A (112 aa) +  input sequence B (116 aa) = 228 aa


In [ ]:
# take a look at the scores
plddt_scores

If we take a look at the min and max values of the pLDDT scores, we can see that the scores are between 0 and 1. 

In [ ]:

print(max(plddt_scores), min(plddt_scores))

Optionally, for consistency with the ESMFold method, we will multiply the pLDDT scores by 100 to get the scores between 0 and 100. This line of code is shown in the `load_protein_boltz` function later in the nobeook. 

## Visualizing the results with py3Dmol

From what we have seen above, to visaulize the protein, we need to match the pLDDT scores to the correct residues in the PDB file. 

Now open the `multimer_model_0.pdb` file and take a look at the raw PDB file. 

Start counting columns from 0: 

- `column 1`: atom number, starting from 1
- `Column 4`: this the chain ID: A or B
- `Column 5`: this is the residue number counting from 1 to 112 for chain A and 1 to 116 for chain B

This tells us our strategy could be: 
- read the PDB file line by line
- split each line by column
- Look at `column 1` (0 indexed), if it is not "ATOM", ignore the line
- Extract the residue number from `column 5` (0 indexed), and match it to the pLDDT score
- Style each atom in this residue based on the pLDDT score

Based on this, we will rewrite the `load_protein_boltz` function to visualize the results. 

In [8]:
import py3Dmol
def load_protein_boltz(pdb_file_path, plddt_file_path, width=800, height=600):

    """
    Load a protein structure from a PDB file and display it using py3Dmol
    pdb_file_path: str, path to the PDB file
    plddt_file_path: str, path to the npz file containing the pLDDT scores
    width: int, width of the viewer in pixels
    height: int, height of the viewer in pixels
    return: py3Dmol.view object
    """
    
    # load the pdb file
    with open(pdb_file_path) as ifile:
        pdb_data = "".join([x for x in ifile])

    # load the plddt scores
    scores = np.load(plddt_file_path)['plddt']
    
    view = py3Dmol.view(width=width, height=height)
    view.addModelsAsFrames(pdb_data)
    

    for line in pdb_data.split("\n"):

        # split each line by columns
        split = line.split()

        # not a valid line, ignore
        if len(split) == 0 or split[0] != "ATOM":
            continue

        # get residue id, pdb is 1-indexed, python is 0-indexed, therefore -1 to convert
        residue_idx = int(split[5]) - 1 

        # get the pLDDT score for the current residue, scale it to 0-100
        plddt_score = scores[residue_idx] * 100

        if plddt_score > 90:
            color = "blue"
        elif 70 <= plddt_score <= 90:
            color = "cyan"
        elif 50 <= plddt_score < 70:
            color = "yellow"
        else:
            color = "orange"
        
        # Atom serial numbers typically start from 1, similar to requirement of `view.setStyle`, hence idx should be used directly
        idx = int(split[1])
        
        # Style should be set per atom id
        view.setStyle({'model': -1, 'serial': idx}, {"cartoon": {'color': color}})
    view.zoomTo()
    return view


In [ ]:
view = load_protein_boltz(
    pdb_file_path="output/boltz_results_multimer/predictions/multimer/multimer_model_0.pdb", 
    plddt_file_path="output/boltz_results_multimer/predictions/multimer/plddt_multimer_model_0.npz")
view
